In [3]:
import os                      # 운영체제 기능 사용을 위한 모듈(파일 존재 확인 등)
import pickle                  # 파이썬 객체를 바이너리로 직렬화/역직렬화하기 위한 모듈
import re                      # 정규식(문자열 패턴 검사) 사용을 위한 모듈

FILE_NAME = "members.dat"      # 회원 데이터를 저장할 바이너리 파일 이름 지정

members = []                   # 프로그램 실행 중 사용할 회원 리스트(각 항목은 dict)

# --- 파일 로드/저장 ---
def load_file():               # 저장된 파일이 있으면 불러오는 함수 정의 시작
    global members             # 전역 변수 members를 함수 내부에서 사용/수정하겠다고 선언
    if os.path.exists(FILE_NAME):            # 파일이 존재하는지 확인
        with open(FILE_NAME, "rb") as f:     # 파일을 바이너리 읽기 모드로 열기
            members = pickle.load(f)         # pickle로 저장된 객체를 불러와 members에 할당
    else:                                   # 파일이 없을 경우
        members = []                         # 빈 리스트로 초기화

def save_file():               # 현재 members를 파일에 저장하는 함수 정의 시작
    with open(FILE_NAME, "wb") as f:         # 파일을 바이너리 쓰기 모드로 열기
        pickle.dump(members, f)              # members 객체를 pickle로 파일에 기록

# --- 유효성 검사 함수 ---
def valid_name(name):          # 이름 형식 검사 함수 정의 (반환값: True/False)
    return 1 <= len(name) <= 5 and name.isalnum()   # 길이 1~5, 영문/한글/숫자 조합 허용

def valid_phone(phone):        # 전화번호 형식 검사 함수 정의
    return re.match(r"^010-\d{4}-\d{4}$", phone) is not None  # 000-0000-0000 형태 검사

def valid_rel(rel):            # 관계 입력(코드) 유효성 검사 함수 정의
    return rel in ["1", "2", "3"]   # 허용값은 "1", "2", "3" 만

def valid_addr(addr):          # 주소 길이 검사 함수 정의
    return len(addr) <= 100     # 최대 100자 이내여야 함

def phone_exists(phone, exclude_index=None):   # 전화번호 중복 검사 함수 정의
    for i, m in enumerate(members):            # members를 순회하면서 인덱스와 항목을 얻음
        if m["phone"] == phone and i != exclude_index:  # 같은 전화번호가 있고, 제외 인덱스가 아니면
            return True                         # 중복이므로 True 반환
    return False                                # 중복 없으면 False 반환

# --- 기능 ---
def list_members():            # 회원 목록을 출력하는 함수 정의
    if not members:            # members가 빈 리스트이면
        print("회원이 없습니다.")  # 회원 없음 메시지 출력
        return                 # 함수 종료
    print("\n[회원 목록]")      # 목록 제목 출력
    for idx, m in enumerate(members, 1):   # 인덱스(1부터)와 회원 dict를 순회
        print(f"{idx}. 이름:{m['name']}, 전화:{m['phone']}, 관계:{m['rel']}, 주소:{m['addr']}")  # 항목 출력
    print(f"총 인원: {len(members)}\n")    # 총 회원 수 출력

def add_member():              # 회원 추가 기능 함수 정의
    name = input("이름 (1~5자): ")   # 이름 입력 받음
    if not valid_name(name):          # 이름 유효성 검사
        print("이름 형식 오류")        # 오류 메시지 출력
        return                         # 함수 종료

    phone = input("전화번호 (000-000-000): ")  # 전화번호 입력 받음
    if not valid_phone(phone):                # 전화번호 형식 검사
        print("전화번호 형식 오류")            # 오류 메시지 출력
        return                                 # 함수 종료
    if phone_exists(phone):                    # 전화번호 중복 검사
        print("이미 등록된 전화번호입니다.")     # 중복 시 메시지 출력
        return                                 # 함수 종료

    rel = input("관계 (1가족,2친구,3기타): ")   # 관계 코드 입력 받음
    if not valid_rel(rel):                     # 관계 유효성 검사
        print("관계 입력 오류")                 # 오류 메시지 출력
        return                                  # 함수 종료

    addr = input("주소 (100자 이내): ")         # 주소 입력 받음
    if not valid_addr(addr):                    # 주소 길이 검사
        print("주소 입력 오류")                  # 오류 메시지 출력
        return                                  # 함수 종료

    members.append({"name": name, "phone": phone, "rel": rel, "addr": addr})  # members에 dict 추가
    print("저장 완료!")                         # 저장 성공 메시지 출력

def modify_member():           # 회원 수정 기능 함수 정의
    name = input("수정할 회원 이름: ")     # 수정 대상 이름 입력 받음
    found = [m for m in members if m["name"] == name]  # 이름으로 검색하여 리스트 생성
    if not found:                          # 검색 결과가 없으면
        print("해당 이름의 회원 없음")        # 없다는 메시지 출력
        return                             # 함수 종료

    list_members()                         # 전체 목록 출력(인덱스 확인용)
    try:
        idx = int(input("수정할 번호: ")) - 1   # 수정할 번호 입력받아 0-based 인덱스로 변환
        if idx < 0 or idx >= len(members):      # 범위 검사
            print("잘못된 번호")                 # 잘못된 번호 메시지
            return                              # 함수 종료
    except ValueError:                           # 숫자 변환 실패 시 처리
        print("숫자만 입력하세요")               # 안내 메시지 출력
        return                                  # 함수 종료

    new_name = input("새 이름 (1~5자): ")        # 새 이름 입력
    if not valid_name(new_name):                 # 새 이름 유효성 검사
        print("이름 오류")                        # 오류 메시지 출력
        return                                   # 함수 종료

    new_phone = input("새 전화번호 (000-0000-0000): ")  # 새 전화번호 입력
    if not valid_phone(new_phone) or phone_exists(new_phone, exclude_index=idx):  # 형식/중복 검사(자기 자신 제외)
        print("전화번호 오류 또는 중복")            # 오류/중복 메시지 출력
        return                                    # 함수 종료

    new_rel = input("새 관계 (1,2,3): ")           # 새 관계 입력
    if not valid_rel(new_rel):                    # 새 관계 유효성 검사
        print("관계 오류")                         # 오류 메시지 출력
        return                                     # 함수 종료

    new_addr = input("새 주소 (100자 이내): ")     # 새 주소 입력
    if not valid_addr(new_addr):                   # 주소 길이 검사
        print("주소 오류")                          # 오류 메시지 출력
        return                                      # 함수 종료

    members[idx] = {
        "name": new_name,
        "phone": new_phone,
        "rel": new_rel,
        "addr": new_addr
    }                                             # 선택한 인덱스의 데이터 수정
    print("수정 완료!")                            # 수정 완료 메시지 출력

def delete_member():           # 회원 삭제 기능 함수 정의
    name = input("삭제할 회원 이름: ")     # 삭제 대상 이름 입력 받음
    found = [m for m in members if m["name"] == name]  # 이름으로 검색
    if not found:                          # 검색 결과가 없으면
        print("해당 이름의 회원 없음")        # 없음 메시지 출력
        return                             # 함수 종료

    list_members()                         # 전체 목록 출력(번호 확인)
    try:
        idx = int(input("삭제할 번호: ")) - 1   # 삭제할 번호 입력받아 0-based로 변환
        if idx < 0 or idx >= len(members):      # 범위 검사
            print("잘못된 번호")                 # 잘못된 번호 메시지 출력
            return                              # 함수 종료
    except ValueError:                           # 숫자 입력이 아닐 경우
        print("숫자만 입력하세요")               # 안내 메시지 출력
        return                                  # 함수 종료

    confirm = input("정말 삭제하시겠습니까? (y/n): ")  # 최종 삭제 확인 입력
    if confirm.lower() == "y":                          # 'y' 또는 'Y'면 삭제 수행
        del members[idx]                                # 해당 인덱스의 회원 데이터 삭제
        print("삭제 완료!")                              # 삭제 완료 메시지 출력
    else:                                               # 확인이 'y'가 아니면
        print("취소됨")                                  # 취소 메시지 출력

# --- 메인 루프 ---
def main():                   # 메인 프로그램 루프 함수 정의
    load_file()               # 시작 시 저장된 파일이 있으면 불러옴
    while True:               # 무한 루프 시작(사용자 종료 선택 시까지)
        print("\n[메뉴] 1.목록 2.추가 3.수정 4.삭제 5.종료")  # 메뉴 출력
        choice = input("번호 입력: ")       # 사용자 선택 입력 받음

        if choice == "1":                   # 선택이 1이면
            list_members()                  # 목록 출력 함수 호출
        elif choice == "2":                 # 선택이 2이면
            add_member()                    # 추가 함수 호출
        elif choice == "3":                 # 선택이 3이면
            modify_member()                 # 수정 함수 호출
        elif choice == "4":                 # 선택이 4이면
            delete_member()                 # 삭제 함수 호출
        elif choice == "5":                 # 선택이 5이면 종료 절차
            confirm = input("종료하시겠습니까? (y/n): ")  # 종료 확인 입력
            if confirm.lower() == "y":                  # 'y'이면 저장 후 종료
                save_file()                              # 파일에 저장
                print("저장 후 종료합니다.")              # 종료 메시지 출력
                break                                   # 루프 탈출(프로그램 종료)
        else:                                          # 1~5가 아닌 다른 입력일 경우
            print("올바른 번호를 입력하세요.")           # 경고 메시지 출력

if __name__ == "__main__":     # 이 파일을 직접 실행할 때만 아래 실행
    main()                     # 메인 함수 실행



[메뉴] 1.목록 2.추가 3.수정 4.삭제 5.종료
올바른 번호를 입력하세요.

[메뉴] 1.목록 2.추가 3.수정 4.삭제 5.종료

[메뉴] 1.목록 2.추가 3.수정 4.삭제 5.종료
저장 후 종료합니다.
